In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import cv2

In [12]:
class OtsuThresholding(object):
    def __init__(self):
        pass
    
    def __call__(self, img):
        # Convert PIL Image or Tensor to NumPy array (if not already)
        if isinstance(img, torch.Tensor):
            img = img.numpy().squeeze(0)  # Remove the channel dimension if grayscale
            img = (img * 255).astype(np.uint8)  # Convert to uint8
        
        # Apply Otsu's Thresholding
        _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Convert back to Tensor
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0) / 255.0
        return img

In [2]:
transform = transforms.Compose([
    transforms.Grayscale(),  # Chuyển đổi sang ảnh xám trước khi áp dụng Otsu's Thresholding
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [3]:
def is_valid_image(path):
    try:
        img = Image.open(path)
        img.verify()
        return True
    except (IOError, SyntaxError) as e:
        print(f"Bad file: {path}")
        return False

In [ ]:
train_dataset = datasets.ImageFolder(root='OCR\\gray_data\\training_data', transform=transform, is_valid_file=is_valid_image)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageFolder(root='OCR\\gray_data\\test_data', transform=transform, is_valid_file=is_valid_image)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

test_dataset = datasets.ImageFolder(root='OCR\\data2\\testing_data', transform=transform, is_valid_file=is_valid_image)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64*8*8, 128)
        self.fc2 = nn.Linear(128, 36)  # 36 output classes (26 letters + 10 digits)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 64*8*8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
def trainModel(model):
    num_epochs = 40
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)  # Chuyển images sang cùng thiết bị với model
            labels = labels.to(device)  # Chuyển labels sang cùng thiết bị với model
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass và optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')
        torch.save(model.state_dict(), f'model_epoch_{epoch+1}.pth')
        
        # Đánh giá mô hình sau mỗi epoch
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        print(f'Validation Accuracy: {100 * correct / total} %')

print("Done!")


Done!


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = SimpleCNN().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

cuda


In [19]:
trainModel(model)

Epoch [1/40], Loss: 0.34074352481632686
Validation Accuracy: 97.14285714285714 %
Epoch [2/40], Loss: 0.0926566188913611
Validation Accuracy: 99.18367346938776 %
Epoch [3/40], Loss: 0.053430174961557095
Validation Accuracy: 99.08163265306122 %
Epoch [4/40], Loss: 0.032633934612761914
Validation Accuracy: 99.79591836734694 %
Epoch [5/40], Loss: 0.02051191053373987
Validation Accuracy: 99.59183673469387 %
Epoch [6/40], Loss: 0.018111307667878832
Validation Accuracy: 99.59183673469387 %
Epoch [7/40], Loss: 0.019380988547804432
Validation Accuracy: 100.0 %
Epoch [8/40], Loss: 0.01782402531026046
Validation Accuracy: 99.79591836734694 %
Epoch [9/40], Loss: 0.007359715120148611
Validation Accuracy: 99.89795918367346 %
Epoch [10/40], Loss: 0.01112286682637299
Validation Accuracy: 99.89795918367346 %
Epoch [11/40], Loss: 0.011418974068560695
Validation Accuracy: 99.48979591836735 %
Epoch [12/40], Loss: 0.010624879602793433
Validation Accuracy: 100.0 %
Epoch [13/40], Loss: 0.007246180464627398
V

### load lại model đã lưu

In [5]:
model = SimpleCNN().to('cuda')

model.load_state_dict(torch.load('model_epoch_40.pth'))


C:\Users\Hokta\AppData\Local\Temp\ipykernel_9996\2006280369.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_epoch_40.pth'))


<All keys matched successfully>

In [24]:
alphabet_and_digit = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [27]:
def testModel(model, test_loader, device):
    model.eval()  # Đặt mô hình ở chế độ đánh giá
    correct = 0
    total = 0
    incorrect_predictions = []  # Danh sách lưu trữ các dự đoán sai

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Lưu trữ các dự đoán sai
            for i in range(labels.size(0)):
                if predicted[i] != labels[i]:
                    incorrect_predictions.append((predicted[i].item(), labels[i].item()))

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy} %')

    print(f'The accuracy: {correct} / {total} \n')
    # In ra các dự đoán sai
    if incorrect_predictions:
        print("\nIncorrect predictions:")
        for pred, label in incorrect_predictions:
            print(f"Predicted: {alphabet_and_digit[pred]}, Actual: {alphabet_and_digit[label]}")
    else:
        print("No incorrect predictions!")

# Gọi hàm testModel
testModel(model, test_loader, device)


Test Accuracy: 98.858592263792 %
The accuracy: 3118 / 3154 


Incorrect predictions:
Predicted: I, Actual: 0
Predicted: I, Actual: 0
Predicted: I, Actual: 0
Predicted: I, Actual: 0
Predicted: 1, Actual: 2
Predicted: I, Actual: A
Predicted: 4, Actual: A
Predicted: 3, Actual: B
Predicted: 3, Actual: B
Predicted: I, Actual: G
Predicted: I, Actual: G
Predicted: F, Actual: H
Predicted: F, Actual: H
Predicted: 1, Actual: J
Predicted: I, Actual: J
Predicted: I, Actual: J
Predicted: I, Actual: J
Predicted: I, Actual: J
Predicted: 1, Actual: L
Predicted: 1, Actual: L
Predicted: V, Actual: M
Predicted: V, Actual: M
Predicted: 1, Actual: N
Predicted: Y, Actual: N
Predicted: R, Actual: P
Predicted: P, Actual: R
Predicted: P, Actual: R
Predicted: H, Actual: R
Predicted: 5, Actual: S
Predicted: J, Actual: U
Predicted: J, Actual: U
Predicted: U, Actual: V
Predicted: 1, Actual: X
Predicted: 1, Actual: X
Predicted: V, Actual: Y
Predicted: I, Actual: Z


In [13]:
def predict_image(image_path):
    # Đảm bảo mô hình ở chế độ đánh giá
    model.eval()
    
    # Đọc ảnh từ đường dẫn
    image = Image.open(image_path)
    
    # Áp dụng các phép biến đổi cho ảnh
    image = transform(image).unsqueeze(0)  # Thêm batch dimension
    
    # Chuyển ảnh và mô hình sang thiết bị (CPU/GPU)
    image = image.to(device)
    model.to(device)
    
    # Tiến hành dự đoán
    with torch.no_grad():
        outputs = model(image)
    
    # Chọn lớp có xác suất cao nhất
    _, predicted = torch.max(outputs, 1)
    
    # Chuyển đổi chỉ số dự đoán thành nhãn lớp
    # Giả sử bạn có danh sách các lớp
    classes =  [0, 1, 2, 3, 4,5, 6, 7, 8, 9, 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'
]
    predicted_class = classes[predicted.item()]
    
    return predicted_class

In [22]:
# Đọc ảnh bằng PIL
image = Image.open('plates.png')

# Chuyển ảnh từ PIL sang NumPy array
image_np = np.array(image)

# Chuyển đổi ảnh màu (BGR) sang ảnh xám
gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

# Hiển thị ảnh xám
cv2.imshow('Gray Image', gray)

# Đợi nhấn phím để đóng cửa sổ
cv2.waitKey(0)

# Đóng tất cả các cửa sổ
cv2.destroyAllWindows()

In [4]:
def find_and_draw_contours():
    # Đọc ảnh đã threshold (ảnh xám)
    preprocessed_image = cv2.imread('thresholded_image.jpg', cv2.IMREAD_GRAYSCALE)
    original_image = cv2.imread('resized_image.jpg')

    # Tìm các đường viền trên ảnh đã được threshold
    contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        
        # Điều kiện lọc kích thước để loại bỏ các vùng không phải là đối tượng cần thiết
        if h > 10 and w > 10:
            # Vẽ hình chữ nhật xung quanh vùng đối tượng
            cv2.rectangle(preprocessed_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Hiển thị ảnh với các đường viền
    cv2.imshow("Contours", preprocessed_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Gọi hàm để thực hiện phát hiện đường viền
find_and_draw_contours()

In [9]:
import numpy as np
def segment_objects(image_path):
    # Đọc ảnh màu xám
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Áp dụng ngưỡng hóa
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Tạo kernel để thực hiện morphological operations
    kernel = np.ones((3, 3), np.uint8)

    # Áp dụng opening để loại bỏ nhiễu nhỏ
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Dilation để chắc chắn rằng các đối tượng đã được tách biệt
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Distance Transform để tìm khoảng cách từ các điểm foreground
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Chuyển đổi foreground thành kiểu số nguyên
    sure_fg = np.uint8(sure_fg)

    # Tìm các vùng không chắc chắn
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Tạo markers cho thuật toán Watershed
    _, markers = cv2.connectedComponents(sure_fg)

    # Thêm 1 vào tất cả các markers để đảm bảo background là 1
    markers = markers + 1

    # Đặt vùng không chắc chắn là 0
    markers[unknown == 0] = 0

    # Áp dụng Watershed
    original_image = cv2.imread(image_path)
    markers = cv2.watershed(original_image, markers)

    # Vẽ các đường viền tách biệt
    original_image[markers == -1] = [0, 255, 0]

    # Hiển thị ảnh kết quả
    cv2.imshow("Segmented Objects", original_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Gọi hàm với đường dẫn ảnh
segment_objects('images\\test\\3.jpg')

In [10]:
import cv2
import numpy as np

# Đọc ảnh nhị phân
image = cv2.imread('images\\test\\3.jpg', cv2.IMREAD_GRAYSCALE)

# Tạo kernel
kernel = np.ones((5, 5), np.uint8)

# Erosion
eroded_image = cv2.erode(image, kernel, iterations=1)

# Dilation
dilated_image = cv2.dilate(image, kernel, iterations=1)

# Opening
opening_image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# Closing
closing_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)

# Morphological Gradient
gradient_image = cv2.morphologyEx(image, cv2.MORPH_GRADIENT, kernel)

# Top Hat
tophat_image = cv2.morphologyEx(image, cv2.MORPH_TOPHAT, kernel)

# Black Hat
blackhat_image = cv2.morphologyEx(image, cv2.MORPH_BLACKHAT, kernel)

# Hiển thị kết quả
cv2.imshow('Original', image)
cv2.imshow('Eroded', eroded_image)
cv2.imshow('Dilated', dilated_image)
cv2.imshow('Opening', opening_image)
cv2.imshow('Closing', closing_image)
cv2.imshow('Gradient', gradient_image)
cv2.imshow('Top Hat', tophat_image)
cv2.imshow('Black Hat', blackhat_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [25]:
import cv2
import numpy as np

# Đọc ảnh và chuyển về ảnh xám
image = cv2.imread('1.jpg', cv2.IMREAD_GRAYSCALE)

# Áp dụng thresholding để tạo ảnh nhị phân
# _, binary_image = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY)
_, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Sử dụng Connected Components
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_image, connectivity=8)

# Tạo ảnh màu để hiển thị các thành phần liên thông
output_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

# Gán màu ngẫu nhiên cho mỗi thành phần liên thông (trừ nền)
for i in range(1, num_labels):
    mask = labels == i
    output_image[mask] = np.random.randint(0, 255, size=(3,), dtype=np.uint8)

# Hiển thị kết quả
cv2.imshow('Connected Components', output_image)
cv2.waitKey(0)

cv2.imshow('original image', image)
cv2.waitKey(0)

cv2.imshow('Components', binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [10]:
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

def predict_image(image):
    # Đọc ảnh
    # image = Image.open(image_path).convert('RGB')

    # Tiền xử lý ảnh
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)  # Thêm chiều batch

    # Kiểm tra xem có GPU không
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    # Dự đoán
    with torch.no_grad():
        output = model(input_batch)
    
    # Tinh toán xác suất và lớp dự đoán
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    top_prob, top_class = torch.topk(probabilities, 1)
    
    # In kết quả
    predicted_class = classes[top_class[0].item()]
    predicted_prob = top_prob[0].item()
    
    print(f'Predicted class: {predicted_class}')
    print(f'Probability: {predicted_prob:.4f}')
    return predicted_class

In [20]:
predict_image('output\\bboxes\\bbox_2_label_U_conf_0.92.jpg')  # Thay đổi đường dẫn đến ảnh của bạn


Predicted class: D
Probability: 0.6855
